In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Data Exploration

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [39]:
path = "cifar-10-batches-py/data_batch_1"
data = unpickle(path)


testPath = "cifar-10-batches-py/test_batch"
testData = unpickle(testPath)

In [41]:
labels = data[list(data.keys())[1]]
images = data[list(data.keys())[2]]

testLabels = testData[list(testData.keys())[1]]
testImages = testData[list(testData.keys())[2]]

In [37]:
pic = 1

print(labels[pic])
images[pic].reshape(32,32,3)

9


array([[[154, 126, 105],
        [102, 125, 155],
        [172, 180, 142],
        ...,
        [ 88, 103,  94],
        [ 65,  83,  90],
        [ 79,  68,  67]],

       [[136, 137, 122],
        [132, 151, 181],
        [203, 208, 208],
        ...,
        [ 92,  88,  78],
        [ 87,  98,  76],
        [ 67,  81,  91]],

       [[146, 124,  88],
        [ 85,  87,  84],
        [ 75,  78,  69],
        ...,
        [169, 113,  89],
        [ 84,  65,  56],
        [ 88,  81,  63]],

       ...,

       [[158,  83,  55],
        [ 46,  51,  52],
        [ 48,  46,  57],
        ...,
        [134, 121,  61],
        [ 51,  33,  21],
        [ 53,  51, 107]],

       [[172, 166, 123],
        [140, 160, 155],
        [139, 131, 126],
        ...,
        [ 96, 101, 135],
        [ 87,  78,  29],
        [ 84,  73,  94]],

       [[166, 160, 170],
        [163, 165, 171],
        [180, 186, 174],
        ...,
        [ 42,  67, 101],
        [122, 133, 136],
        [139, 142, 144]]

# Data Processing

In [86]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [100]:
scaler = StandardScaler()

scaler.fit(images)

trainX = scaler.transform(images)
testX = scaler.transform(testImages)

encoder = OneHotEncoder()
encoder.fit(np.array(labels).reshape(-1, 1))

trainY = encoder.transform(np.array(labels).reshape(-1, 1)).toarray()
testY = encoder.transform(np.array(testLabels).reshape(-1, 1)).toarray()

In [101]:
#Reshape data for CNN
trainX = trainX[..., None]

testX = testX[..., None]


# Model Creation

In [97]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras import layers, models
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import tensorflow as tf

In [144]:
batch_size = 128
num_epochs = 10
lr = .01
    
model = Sequential()

model.add((Conv1D(filters=64, kernel_size=5, activation='relu')))
model.add((Conv1D(filters=32, kernel_size=4, activation='relu')))
model.add((Conv1D(filters=16, kernel_size=3, activation='relu')))
model.add((MaxPooling1D(pool_size=2)))
model.add((Conv1D(filters=8, kernel_size=2, activation='relu')))
model.add((MaxPooling1D(pool_size=2)))
model.add((Conv1D(filters=4, kernel_size=3, activation='relu')))
model.add((MaxPooling1D(pool_size=2)))

#output
model.add(Flatten())
model.add(Dense(10, activation='sigmoid'))

adam = tf.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=10e-7, decay=0.01)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics = ["Precision","Recall"])

In [ ]:
model.fit(trainX, trainY, epochs = num_epochs, batch_size=batch_size)

Epoch 1/10
 8/79 [==>...........................] - ETA: 1:04 - loss: 2.3015 - precision: 0.1112 - recall: 0.4766

In [138]:
model.evaluate(testX,testY)

313/313 [==============================] - 11s 33ms/step - loss: 1.5775 - precision: 0.3718 - recall: 0.5402


[1.5775079727172852, 0.37180811166763306, 0.5401999950408936]

In [139]:
pred = model.predict(testX)

In [140]:
error = 0

for i in range(0,len(pred)):
    if np.argmax(pred[i]) != testLabels[i]:
        error += 1


acc = 1 - (error/len(pred))
print(acc)

0.4617
